# cadCAD Model Creation

This notebook shows how to build an actual cadCAD model (from the spec builder side) versus the other cadCAD notebook shows how cadCAD models could be handed off to a data scientist with no need to build them.

## Initialize Mathematical Specification

In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from simulation import (params_base, state_base, compute_starting_total_length, check_d_probability, post_processing_function,
                        percent_ending_in_d_metric, average_d_count_metric, plot_length_single_simulation,)


from math_spec_mapping import load_from_json, get_nested_types
#sys.path.append(os.path.abspath('../..'))
#from MSML.src.math_spec_mapping import (load_from_json, get_nested_types)

from copy import deepcopy
from src import math_spec_json

ms = load_from_json(deepcopy(math_spec_json), spec_path="../src")

## Create a cadCAD Simulation

In [2]:
blocks = ["DUMMY Length-2 Boundary Wiring",
               "DUMMY Length-1 Boundary Wiring",
               "DUMMY Control Wiring"]

state_space, parameter_space, model = (ms.build_cadCAD(blocks,
                  state_preperation_functions=[compute_starting_total_length],
                  parameter_preperation_functions=[check_d_probability],)).values()

Display the nested types involved in the state space and parameter space

In [3]:
display(get_nested_types(state_space))
print()
display(get_nested_types(parameter_space))

{'Dummy': {'Words': str, 'Total Length': int},
 'Time': int,
 'Simulation Log': [{'Time': int, 'Word': str, 'Length (Multiplied)': int}]}

{'DUMMY D Probability': float,
 'DUMMY Length Multiplier': int,
 'FP DUMMY Length-2 ABC Combo Boundary Action': typing.Literal['DUMMY Length-2 ABC Equal Weight Option', 'DUMMY Length-2 ABC Equal Weight 2 Option'],
 'FP DUMMY Length-1 DEF Control Action': typing.Literal['DUMMY Length-1 DEF Equal Weight Option', 'DUMMY Length-1 DEF D Probability Option']}

## Data Scientist Interface is the Simulation Object

### Single Experiment

In [4]:
state: state_space = {'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []}
params: parameter_space = {'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'}

experiment = model.create_experiment(state, params, record_trajectory=True, use_deepcopy=True)
experiment.run(5)
trajectories = experiment.trajectories
display(experiment.trajectories[-2:])

[{'Dummy': {'Words': 'BBAEAACEABAFBCBD', 'Total Length': 48},
  'Time': 12,
  'Simulation Log': [{'Time': 1, 'Word': 'BB', 'Length (Multiplied)': 6},
   {'Time': 2, 'Word': 'BBA', 'Length (Multiplied)': 9},
   {'Time': 3, 'Word': 'BBAE', 'Length (Multiplied)': 12},
   {'Time': 4, 'Word': 'BBAEAA', 'Length (Multiplied)': 18},
   {'Time': 5, 'Word': 'BBAEAAC', 'Length (Multiplied)': 21},
   {'Time': 6, 'Word': 'BBAEAACE', 'Length (Multiplied)': 24},
   {'Time': 7, 'Word': 'BBAEAACEAB', 'Length (Multiplied)': 30},
   {'Time': 8, 'Word': 'BBAEAACEABA', 'Length (Multiplied)': 33},
   {'Time': 9, 'Word': 'BBAEAACEABAF', 'Length (Multiplied)': 36},
   {'Time': 10, 'Word': 'BBAEAACEABAFBC', 'Length (Multiplied)': 42},
   {'Time': 11, 'Word': 'BBAEAACEABAFBCB', 'Length (Multiplied)': 45},
   {'Time': 12, 'Word': 'BBAEAACEABAFBCBD', 'Length (Multiplied)': 48}]},
 {'Dummy': {'Words': 'BBAEAACEABAFBCBDCAAF', 'Total Length': 60},
  'Time': 15,
  'Simulation Log': [{'Time': 1, 'Word': 'BB', 'Length 

## Batch Experiments

In [5]:
from typing import List
state: list = [{'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []},
                            {'Dummy': {'Words': '', 'Total Length': None},
                                   'Time': 0,
                                   'Simulation Log': []}]
params: list = [{'DUMMY D Probability': 0.5,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'},
 {'DUMMY D Probability': 0.1,
 'DUMMY Length Multiplier': 3,
 'FP DUMMY Length-1 DEF Control Action': 'DUMMY Length-1 DEF Equal Weight Option',
 'FP DUMMY Length-2 ABC Combo Boundary Action': 'DUMMY Length-2 ABC Equal Weight Option'}]
batch_experiments = model.create_batch_experiments(state, params, record_trajectory=True, use_deepcopy=True)
batch_experiments.run(3)
print(batch_experiments.trajectories)
print(len(batch_experiments.trajectories))
print(len(batch_experiments.trajectories[0]))

[[{'Dummy': {'Words': '', 'Total Length': 0}, 'Time': 0, 'Simulation Log': []}, {'Dummy': {'Words': 'CCBD', 'Total Length': 12}, 'Time': 3, 'Simulation Log': [{'Time': 1, 'Word': 'CC', 'Length (Multiplied)': 6}, {'Time': 2, 'Word': 'CCB', 'Length (Multiplied)': 9}, {'Time': 3, 'Word': 'CCBD', 'Length (Multiplied)': 12}]}, {'Dummy': {'Words': 'CCBDBCBD', 'Total Length': 24}, 'Time': 6, 'Simulation Log': [{'Time': 1, 'Word': 'CC', 'Length (Multiplied)': 6}, {'Time': 2, 'Word': 'CCB', 'Length (Multiplied)': 9}, {'Time': 3, 'Word': 'CCBD', 'Length (Multiplied)': 12}, {'Time': 4, 'Word': 'CCBDBC', 'Length (Multiplied)': 18}, {'Time': 5, 'Word': 'CCBDBCB', 'Length (Multiplied)': 21}, {'Time': 6, 'Word': 'CCBDBCBD', 'Length (Multiplied)': 24}]}, {'Dummy': {'Words': 'CCBDBCBDAABF', 'Total Length': 36}, 'Time': 9, 'Simulation Log': [{'Time': 1, 'Word': 'CC', 'Length (Multiplied)': 6}, {'Time': 2, 'Word': 'CCB', 'Length (Multiplied)': 9}, {'Time': 3, 'Word': 'CCBD', 'Length (Multiplied)': 12}, {

## Appendix A. Show Detailed State and Parameter Tables using MSML functionality

In [8]:
from math_spec_mapping import write_initial_state_variables_tables, write_parameter_table_markdown
from IPython.display import Markdown

table1 = write_initial_state_variables_tables(ms, initial_values=state_base)
table2 = write_parameter_table_markdown(ms, initial_values=params_base)

display(Markdown("# State"))
display(Markdown(table1))
display(Markdown("# Parameters"))
display(Markdown(table2))

# State

### Global State

| Name | Description | Type | Symbol | Domain | Initial Value |
| --- | --- | --- | --- | --- | --- |
|Dummy|The dummy entity|Entity Type||| {'Words': '', 'Total Length': None} |
|Time|The clock time|DUMMY Integer Type||| 0 |
|Simulation Log|The simulation log holding historical data|Simulation Log Type||| [] |

### DUMMY State

| Name | Description | Type | Symbol | Domain |
| --- | --- | --- | --- | --- |
|Words|All words that were created|DUMMY ABCDEF Type|||
|Total Length|The total length of words * multiplier|DUMMY Integer Type|||



# Parameters

| Name | Description | Parameter Class | Symbol | Domain | Initial Value |
| --- | --- | --- | --- | --- | --- |
|DUMMY D Probability|The probability that D is chosen|Behavioral||[0, 1]| 0.5 |
|DUMMY Length Multiplier|A multiplier to multiply into length calculations|System||$\mathbb{Z}$| 3 |
